In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.read_csv("E:\\NLP\\train.csv")

In [3]:
df.shape

(20800, 5)

In [4]:
df.columns

Index(['id', 'title', 'author', 'text', 'label'], dtype='object')

In [5]:
df=df.drop(columns=['id','author'],axis=1)

In [6]:
df.columns

Index(['title', 'text', 'label'], dtype='object')

In [7]:
df.isnull().sum()

title    558
text      39
label      0
dtype: int64

In [8]:
df=df.dropna()

In [9]:
df.isnull().sum()

title    0
text     0
label    0
dtype: int64

In [10]:
X=df.drop('label',axis=1)
y=df['label']

In [11]:
y.value_counts() # data is not balanced

0    10387
1     9816
Name: label, dtype: int64

In [13]:
import tensorflow

In [14]:
tensorflow.__version__

'2.10.0'

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [18]:
voc_size=5000

In [19]:
messages=X.copy()

In [20]:
messages.reset_index(inplace=True)

In [26]:
import nltk
import re
from nltk.corpus import stopwords

In [27]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
# data preprocessing
from nltk.stem.porter import PorterStemmer 
ps=PorterStemmer()
corpus=[]
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [29]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [30]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr[10]   

[508, 618, 878, 4166, 3240, 744, 3545, 140, 1886, 40]

In [31]:
corpus[10]

'obama organ action partner soro link indivis disrupt trump agenda'

In [36]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs[10])
print(corpus[10])

[   0    0    0    0    0    0    0    0    0    0  508  618  878 4166
 3240  744 3545  140 1886   40]
obama organ action partner soro link indivis disrupt trump agenda


In [37]:
# creating model
embedding_vector_features=40 # feature representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirectiona  (None, 200)              112800    
 l)                                                              
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [38]:
len(embedded_docs),y.shape

(20203, (20203,))

In [39]:
X_final=np.array(embedded_docs)
y_final=np.array(y) 

In [40]:
X_final.shape,y_final.shape

((20203, 20), (20203,))

In [41]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [45]:
# final training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10, batch_size=1000)

Epoch 1/10
14/14 [==============================] - 25s 2s/step - loss: 0.4695 - accuracy: 0.7978 - val_loss: 0.2902 - val_accuracy: 0.8847
Epoch 2/10
14/14 [==============================] - 29s 2s/step - loss: 0.2169 - accuracy: 0.9203 - val_loss: 0.2379 - val_accuracy: 0.9156
Epoch 3/10
14/14 [==============================] - 28s 2s/step - loss: 0.1514 - accuracy: 0.9488 - val_loss: 0.2132 - val_accuracy: 0.9171
Epoch 4/10
14/14 [==============================] - 28s 2s/step - loss: 0.1174 - accuracy: 0.9639 - val_loss: 0.2344 - val_accuracy: 0.9201
Epoch 5/10
14/14 [==============================] - 26s 2s/step - loss: 0.0920 - accuracy: 0.9723 - val_loss: 0.2396 - val_accuracy: 0.9186
Epoch 6/10
14/14 [==============================] - 26s 2s/step - loss: 0.0718 - accuracy: 0.9812 - val_loss: 0.2796 - val_accuracy: 0.9163
Epoch 7/10
14/14 [==============================] - 30s 2s/step - loss: 0.0590 - accuracy: 0.9850 - val_loss: 0.3486 - val_accuracy: 0.9076
Epoch 8/10
14/14 [==

In [46]:
y_pred=model.predict(X_test)

209/209 [==============================] - 6s 27ms/step


In [47]:
y_pred=np.where(y_pred>0.5,1,0)

In [48]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_test,y_pred)

array([[3108,  312],
       [ 290, 2957]], dtype=int64)

In [49]:
print("accuracy prercentage is : {} %".format(((accuracy_score(y_test,y_pred))*100).round(2)))

accuracy prercentage is : 90.97 %


In [50]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3420
           1       0.90      0.91      0.91      3247

    accuracy                           0.91      6667
   macro avg       0.91      0.91      0.91      6667
weighted avg       0.91      0.91      0.91      6667

